In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sys import path
%matplotlib inline

In [2]:
path.insert(0, '../src')
from setup import data_setup
from data_generator import DataGenerator
from model_trainer import ModelTrainer

In [3]:
dg = data_setup()
norm_data = dg.df()

In [6]:
from data_transformation import get_isotope_data, get_isotope_mass_list
isotope_data = get_isotope_data()
nom_masses_low = get_isotope_mass_list(isotope_data, False)
nom_masses_high = get_isotope_mass_list(isotope_data, True)

In [66]:
from data_transformation import get_suspicious_peaks, get_ranges, get_peak_suspiciousness
original_data = dg.calibrated_df(True, .005, .01,use_ranges=True, ranges=[0.25, 0.5, 0.75], cat=True)
ranges = get_ranges(isotope_data, 2000)
original_data['target'] = original_data['target'].apply(lambda a: a - 1 if a > 0 else a)
original_data['sus_peaks'] = original_data['masses'].apply(get_suspicious_peaks, args=(ranges, -1))
original_data['peak_sussness'] = original_data['masses'].apply(get_peak_suspiciousness, args=(ranges, True))

In [68]:
original_data['num_sus_peaks'] = original_data['sus_peaks'].apply(len)

In [69]:
training_data = original_data[original_data['num_sus_peaks'] >= 20].copy()[['peak_sussness', 'num_sus_peaks', 'target']]

In [70]:
def get_20_sussest(susses):
    a = sorted(susses, reverse=True)
    return susses[0:20]

In [78]:
X = np.vstack(training_data['peak_sussness'].apply(get_20_sussest))
X = np.hstack([X, np.array(training_data['num_sus_peaks']).reshape(X.shape[0], 1)])
y = np.array(training_data['target'])

In [80]:
from model_trainer import ModelTrainer
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
models = [RandomForestClassifier, LGBMClassifier, XGBClassifier]
names = ['rfc', 'lgbm', 'xgb']
mt = ModelTrainer(models, X, y, names)

In [81]:
accs, indices_preds = mt.kfold_models(5, model_params=[{'n_estimators': 10}, {}, {}])
accs

[0.8522821049958738, 0.8725385640830319, 0.8703040690662096]